In [2]:
from llm.llm.chatgpt import ChatGPT
from setup_db import create_chroma_db, connect_to_db, execute_query

In [3]:
llm = ChatGPT()

In [6]:
collection_chromadb = 'test_chroma_db'
persist_directory = 'data/test_chroma_db'
vector_store = create_chroma_db(collection_chromadb, persist_directory)

In [2]:
def similarity_search(query, k):
    return [f"'{document.page_content}'" for document in vector_store.similarity_search(query.strip("'"), k)]

In [5]:
from llm_test import text2sql, replace_category_query, CoT_reasoning

In [6]:
prompt = "What is the revenue and the profit of the companies with stock code TCB and VCB in first quarter of 2024"
code_generated = text2sql(llm, prompt)
code_generated

[{'language': 'sql',
  'code': "SELECT * \nFROM financial_statement fs \nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND category IN ('Revenue', 'Profit');\n"}]

In [8]:
for code in code_generated:
    if code['language'] == 'sql':
        code['code'] = replace_category_query(code['code'], similarity_search, top_k=5)
        # vector_store.execute_query(query)

In [9]:
test = "category = 'HYY'"
replace_category_query(test, similarity_search, top_k=5)

"category IN ('Attribute to parent company YoY (%)', 'Revenue YoY (%)', 'Attributable to parent company', 'Attribute to parent company (Bn. VND)', 'Interest and Similar Income')"

In [18]:

# import re

# def top_k(text):
#     # Placeholder for top_k function, replace this with the actual implementation
#     return [f"top_k({text})"]

# def replace_category_query(sql_query):
#     # Pattern to match `category = xx`
#     pattern_equal = re.compile(r"category\s*=\s*'?(\w+)'?")
#     # Pattern to match `category IN (xx, yy)` or `category IN ('xx', 'yy')`
#     pattern_in = re.compile(r"category\s+IN\s*\(([^)]+)\)")
    
#     # Check for `category = xx`
#     if pattern_equal.search(sql_query):
#         sql_query = pattern_equal.sub(lambda match: f"category IN ({', '.join(top_k(match.group(1)))})", sql_query)
    
#     # Check for `category IN (xx, yy)`
#     elif pattern_in.search(sql_query):
#         sql_query = pattern_in.sub(lambda match: f"category IN ({', '.join([item for x in match.group(1).split(',') for item in top_k(x.strip())])})", sql_query)

#     return sql_query

# # Example usage:
# sql_query_1 = "category = 'xx'"
# sql_query_2 = "category IN ('xx', yy)"

# print(replace_category_query(sql_query_1))  # Output for equal size
# print(replace_category_query(sql_query_2))  # Output for IN condition

category IN (top_k(xx))
category IN (top_k('xx'), top_k(yy))


In [10]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
table_name = 'financial_statement'
port = '5433'
host = 'localhost'

In [11]:
code_generated

[{'language': 'sql',
  'code': "SELECT * FROM financial_statement fs \nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND (category IN ('Sales', 'Revenue (Bn. VND)', 'Total operating revenue', 'Revenue YoY (%)', 'Net Sales') OR category IN ('Profit before tax', 'Sales', 'Operating Profit/Loss', 'Gross Profit', 'Net Profit/Loss before tax'));\n"}]

In [12]:
conn = connect_to_db(db_name, user, password, host, port)
result = execute_query(code_generated[0]['code'], conn=conn)
result

Connecting to database test_db, postgres...


[('TCB', 2024, 1, 'Revenue YoY (%)', 0.0564390787932455),
 ('TCB', 2024, 1, 'Revenue (Bn. VND)', 14507238000000.0),
 ('TCB', 2024, 1, 'Total operating revenue', 12261767000000.0),
 ('TCB', 2024, 1, 'Profit before tax', 7801659000000.0),
 ('VCB', 2024, 1, 'Revenue YoY (%)', -0.132547028931519),
 ('VCB', 2024, 1, 'Revenue (Bn. VND)', 24019660000000.0),
 ('VCB', 2024, 1, 'Total operating revenue', 17280398000000.0),
 ('VCB', 2024, 1, 'Profit before tax', 10718239000000.0)]

In [13]:
answer = CoT_reasoning(llm, prompt, result)
print(answer)

To answer the question of what is the revenue and profit of the companies with stock codes TCB and VCB for the first quarter of 2024, we can break down the available information systematically.

1. **Extract Information for TCB:**
   - Looking into the financial statement data provided:
     - For TCB (stock code TCB) in the first quarter of 2024, we find the following figures:
       - "Revenue (Bn. VND)": 14,507,238,000,000.0 VND
       - "Profit before tax": 7,801,659,000,000.0 VND
     
   - **Calculating TCB Revenue and Profit:**
     - Revenue: 14,507,238,000,000.0 VND
     - Profit: 7,801,659,000,000.0 VND

2. **Extract Information for VCB:**
   - Again, looking into the financial data:
     - For VCB (stock code VCB) in the first quarter of 2024, the relevant figures are:
       - "Revenue (Bn. VND)": 24,019,660,000,000.0 VND
       - "Profit before tax": 10,718,239,000,000.0 VND
     
   - **Calculating VCB Revenue and Profit:**
     - Revenue: 24,019,660,000,000.0 VND
     - 

category = xx

category IN (xx, yy)